In [1]:
import os
import sys
import datetime
import numpy as np
import matplotlib.pyplot as plt

parent_directory = os.path.split(os.getcwd())[0]
# sys.path.insert(0, parent_directory)
sys.path.append(parent_directory)

from pandas.core.common import SettingWithCopyWarning
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from gnss_lib_py.io.android import make_gnss_dataframe
from gnss_lib_py.io.nmea import NMEA
from gnss_lib_py.core.measures import FindSat, expected_measures, correct_pseudorange
from gnss_lib_py.utils.constants import GPSConsts
from gnss_lib_py.algorithms.snapshot import solve_pos
from gnss_lib_py.io.android import make_gnss_dataframe
from gnss_lib_py.core.coordinates import ecef2geodetic
from gnss_lib_py.core.ephemeris import EphemerisManager

In [2]:
# e.g. DATA_PATH = "/home/user/data/Google/training/""
DATA_PATH = "/home/derek/datasets/google/training/"

In [3]:
# used for receiver clock bias estimation

def least_squares(x_0,bu,sat_df,pranges):
    """
    input(s)
        x:  [3 x 1] state estimate
        bu: clock bias
        sat_df: satellite data frame
        pranges: psudoranges
    output(s)
        bu: new clock bias
    """
    numSats = len(sat_df)
    dist = np.zeros((numSats,1))

    G = np.zeros((numSats,4))
    W = np.eye(numSats)
    for ii in range(numSats):
        x_s = sat_df["x"].to_numpy()[ii]
        y_s = sat_df["y"].to_numpy()[ii]
        z_s = sat_df["z"].to_numpy()[ii]

        dist[ii] = np.sqrt((x_s-x_0[0])**2 + \
                           (y_s-x_0[1])**2 + \
                           (z_s-x_0[2])**2)
        G[ii,:] = [-(x_s - x_0[0])/dist[ii],
                   -(y_s - x_0[1])/dist[ii],
                   -(z_s - x_0[2])/dist[ii],
                   1.0]
        W[ii,ii] *= 1./pranges[ii]

    rho_0 = dist + bu
    rho_dif = pranges.reshape(-1,1) - rho_0
    delta = np.linalg.pinv(W.dot(G)).dot(W).dot(rho_dif)
    bu_new = bu + delta[3,0]
    return bu_new

In [4]:
def calc_residuals(trace_name, phone_type, verbose = False):
    """Calculate residuals.

    Parameters
    ----------
    trace_name : string
        Provided name for trace, e.g., "2020-05-14-US-MTV-1".
    phone_type : string
        Type of phone used to record data, e.g., "Pixel4", "Pixel4XL",
        "Pixel4XLModded", "Mi8".
    verbose : bool
        If true, prints debugging statements

    """

    print("Analyzing trace ",trace_name,phone_type,"...")

    constants = GPSConsts()

    manager = EphemerisManager(DATA_PATH)

    span_gt_filepath = os.path.join(DATA_PATH, trace_name,
                                    "SPAN_" + phone_type + "_10Hz.nmea")

    # load measurements
    input_filepath = os.path.join(DATA_PATH, trace_name,
                                  phone_type + "_GnssLog.txt")
    measurements, android_fixes = make_gnss_dataframe(input_filepath, True)

    print("Successfully loaded ", measurements.shape[0], " measurements")
    print("Successfully loaded ", android_fixes.shape[0], " android fixes")

    utc0 = int(measurements["utcTimeMillis"].iloc[0])
    measurement_date = datetime.datetime.fromtimestamp(utc0/1000.0)

    # load ground truth
    nmea_obj = NMEA(span_gt_filepath)
    gt_ecef, gt_times, gt_lla = nmea_obj.ecef_gt_w_time(measurement_date.date())

    gt_lla = np.array(gt_lla)
    print("Successfully loaded ",gt_lla.shape[0]," ground truth points.")

    gt_utc_time_millis = np.array(gt_times) * 1000

    # temp shorten dataframe
    gtimes = 1000
    measurements = measurements.head(gtimes)

    # initialize position solution
    nr_lla = []
    nr_ecef = []
    meas_times = []
    gt_ecef_alligned = []
    gt_lla_alligned = []
    receiver_clock_bias = []

    residual_data = {}

    for e, data in measurements.groupby('Epoch'):
        if e % 100 == 0:
            print("e: ",e)
        epoch_all = measurements.loc[(measurements['Epoch'] == e)]

        # drop duplicates
        epoch = epoch_all.drop_duplicates(subset='SvName')

        timestamp = epoch.iloc[0]['UtcTimeNanos'].to_pydatetime(warn=False)
        epoch.set_index('SvName', inplace = True)     # changes dataframe indexing to satellite-based indexing (Really neat!)
        epoch.sort_index(inplace = True) # sorts rows by index
        sats = epoch.index.unique().tolist()
        if len(sats) < 4:
            continue
        ephemeris = manager.get_ephemeris(timestamp, sats)

        # calculate the ground truth location
        meas_time = int(epoch['utcTimeMillis'].values[0])
        gt_ecef_alligned_point = []
        for i in range(3):
            gt_ecef_alligned_point.append(np.interp(meas_time,
                                                    gt_utc_time_millis,
                                                    gt_ecef[:,i]))
        gt_ecef_alligned_point = np.array(gt_ecef_alligned_point)


        # calculate satellite positions
        satpos = FindSat(ephemeris, epoch['tTxSeconds'], epoch['GpsWeekNumber'])
        sat_x = satpos["x"].to_numpy()
        sat_y = satpos["y"].to_numpy()
        sat_z = satpos["z"].to_numpy()

        # get psuedorange
        pranges = epoch['Pseudorange_meters'].to_numpy()
        # correct for satellite clock bias
        pranges = correct_pseudorange(epoch['tTxSeconds'],
                                      epoch['GpsWeekNumber'],
                                      ephemeris,
                                      pranges,
                                      gt_ecef_alligned_point.reshape(1,-1)
                                      )
        sat_b = np.zeros(sat_z.shape)

        # calculate receiver clock bias
        rb = 0.0
        for ii in range(20):
            rb = least_squares(gt_ecef_alligned_point,rb,satpos,pranges)

        # solve for the position
        try:
            pos_ecef = solve_pos(pranges, sat_x, sat_y, sat_z, rb)
        except RuntimeError as err:
            print(err)
            continue

        expected, _ = expected_measures(epoch['GpsWeekNumber'],
                                    epoch['tTxSeconds'],
                                    ephemeris,
                                    gt_ecef_alligned_point.reshape(1,-1),
                                    rb,# 0.0 # -sat_b,
                                    0.0,
                                    np.zeros((1,3)),
                                    satpos)

        gt_psuedoranges = expected["prange"].to_numpy()

        # epoch w/ duplicates removed
        gt_psuedoranges_unique = []
        for index, ss in enumerate(epoch.index):
            gt_psuedoranges_unique.append(gt_psuedoranges[sats.index(ss)])
        epoch.loc[:,"gt_pr"] = gt_psuedoranges_unique

        # calculate residual
        epoch.loc[:,"residual"] = epoch["gt_pr"] \
                                - pranges

        ################################################################
        # RECALCULATE RECEIVER CLOCK BIAS
        ################################################################

        # remove satellites with high residuals to recalculate rb
        residual_mean = epoch["residual"].mean()
        distance_from_mean = abs(epoch["residual"] - residual_mean)
        epoch_rb = epoch[distance_from_mean <= 500]

        sats_rb = epoch_rb.index.unique().tolist()
        if len(sats_rb) < 4:
            continue
        ephemeris_rb = manager.get_ephemeris(timestamp, sats_rb)

        # calculate satellite positions
        satpos_rb = FindSat(ephemeris_rb, epoch_rb['tTxSeconds'], epoch_rb['GpsWeekNumber'])

        # get psuedorange
        pranges_rb = epoch_rb['Pseudorange_meters'].to_numpy()
        # correct for satellite clock bias
        pranges_rb = correct_pseudorange(epoch_rb['tTxSeconds'],
                                      epoch_rb['GpsWeekNumber'],
                                      ephemeris_rb,
                                      pranges_rb,
                                      gt_ecef_alligned_point.reshape(1,-1)
                                      )

        # calculate receiver clock bias
        rb = 0.0
        for ii in range(20):
            rb = least_squares(gt_ecef_alligned_point,rb,satpos_rb,pranges_rb)

        ################################################################
        #
        ################################################################

        # solve for the position
        try:
            pos_ecef = solve_pos(pranges, sat_x, sat_y, sat_z, rb)
        except RuntimeError as err:
            print(err)
            continue

        # append position
        nr_ecef.append(pos_ecef[:3])
        nr_lla.append(ecef2geodetic(pos_ecef[:3]))
        meas_times.append(meas_time)
        gt_ecef_alligned.append(gt_ecef_alligned_point.tolist())
        gt_lla_alligned.append(ecef2geodetic(gt_ecef_alligned_point))

        expected, _ = expected_measures(epoch['GpsWeekNumber'],
                                    epoch['tTxSeconds'],
                                    ephemeris,
                                    gt_ecef_alligned_point.reshape(1,-1),
                                    rb,# 0.0 # -sat_b,
                                    0.0,
                                    np.zeros((1,3)),
                                    satpos)

        gt_psuedoranges = expected["prange"].to_numpy()

        # epoch w/ duplicates removed
        gt_psuedoranges_unique = []
        for index, ss in enumerate(epoch.index):
            gt_psuedoranges_unique.append(gt_psuedoranges[sats.index(ss)])
        epoch.loc[:,"gt_pr"] = gt_psuedoranges_unique

        # calculate residual
        epoch.loc[:,"residual"] = epoch["gt_pr"] \
                                - pranges

        # plot residual data
        for index, ss in enumerate(epoch.index):
            elapsed_sec = (meas_time - meas_times[0]) / 1000.
            if ss not in residual_data:
                residual_data[ss] = [[elapsed_sec,
                                      epoch.loc[ss,"residual"]]]
            else:
                residual_data[ss].append([elapsed_sec,
                                          epoch.loc[ss,"residual"]])

        if verbose:
            print(epoch[["Pseudorange_meters","gt_pr","residual"]])


    nr_lla = np.array(nr_lla)
    gt_lla_alligned = np.array(gt_lla_alligned)

    residual_fig = plt.figure()
    for key, value in residual_data.items():
        value = np.array(value)
        plt.plot(value[:,0],value[:,1],label=key)

    plt.xlabel("time [s]")
    plt.ylabel("residiual [m]")
    plt.legend()

    plt_file = os.path.join(parent_directory, "notebooks", trace_name + "-" \
             + phone_type + ".png")

    residual_fig.savefig(plt_file,
            format="png",
            bbox_inches="tight")

    plt.ylim(-100,200)
    plt_file = os.path.join(parent_directory, "notebooks", trace_name + "-" \
             + phone_type + "-zoomed" + ".png")

    residual_fig.savefig(plt_file,
            format="png",
            bbox_inches="tight")

    plt.close(residual_fig)

In [5]:
def main():
    ephemeris_data_path = DATA_PATH

    # get all trace options
    dirs = sorted(os.listdir(DATA_PATH))
    
    # remove directories that aren't traces
    dirs = [x for x in dirs if x != "igs" and x != "nasa"]

    # create a list of all traces with phone types
    trace_list = []
    for dir in dirs:
        phone_types = []
        trace_path = os.path.join(ephemeris_data_path,dir)
        for file in sorted(os.listdir(trace_path)):
            pt = file.split("_")[0] # potential phone type
            if pt != "SPAN" and pt not in phone_types:
                phone_types.append(pt)

        for phone_type in phone_types:
            trace_list.append((dir,phone_type))

#     trace_list = [
#         ("2020-05-14-US-MTV-1", "Pixel4XLModded"),
#         ("2020-05-14-US-MTV-1", "Pixel4"),
#         #bad("2020-05-14-US-MTV-2", "Pixel4XLModded"),
#         #bad("2020-05-14-US-MTV-2", "Pixel4"),
#         ("2020-05-21-US-MTV-1", "Pixel4XL"),
#         ("2020-05-21-US-MTV-1", "Pixel4"),
#         ("2020-05-21-US-MTV-2", "Pixel4XL"),
#         ("2020-05-21-US-MTV-2", "Pixel4"),
#         ("2020-05-29-US-MTV-1", "Pixel4XLModded"),
#         ("2020-05-29-US-MTV-1", "Pixel4XL"),
#         ("2020-05-29-US-MTV-1", "Pixel4"),
#         #bad("2020-05-29-US-MTV-2", "Pixel4XL"),
#         #bad("2020-05-29-US-MTV-2", "Pixel4"),
#         ("2020-06-04-US-MTV-1", "Pixel4XLModded"),
#         ("2020-06-04-US-MTV-1", "Pixel4XL"),
#         ("2020-06-04-US-MTV-1", "Pixel4"),
#         ("2020-06-05-US-MTV-1", "Pixel4XLModded"),
#         ("2020-06-05-US-MTV-1", "Pixel4XL"),
#         ("2020-06-05-US-MTV-1", "Pixel4"),
#         ("2020-06-05-US-MTV-2", "Pixel4XL"),
#         ("2020-06-05-US-MTV-2", "Pixel4"),
#         #bad("2020-06-11-US-MTV-1", "Pixel4XL"),
#         #bad("2020-06-11-US-MTV-1", "Pixel4"),
#         ("2020-07-08-US-MTV-1", "Pixel4XLModded"),
#         ("2020-07-08-US-MTV-1", "Pixel4XL"),
#         ("2020-07-08-US-MTV-1", "Pixel4"),
#         #bad("2020-07-17-US-MTV-1", "Mi8"),
#         #bad("2020-07-17-US-MTV-2", "Mi8"),
#         ("2020-08-03-US-MTV-1", "Mi8"),
#         ("2020-08-03-US-MTV-1", "Pixel4"),
#         ("2020-08-06-US-MTV-2", "Mi8"),
#         ("2020-08-06-US-MTV-2", "Pixel4XL"),
#         ("2020-08-06-US-MTV-2", "Pixel4"),
#         #bad("2020-09-04-US-SF-1", "Mi8"),
#         ("2020-09-04-US-SF-1", "Pixel4XL"),
#         #bad("2020-09-04-US-SF-1", "Pixel4"),
#         #bad("2020-09-04-US-SF-2", "Mi8"),
#         ("2020-09-04-US-SF-2", "Pixel4XL"),
#         #bad("2020-09-04-US-SF-2", "Pixel4"),
#     ]

    for trace in trace_list:
        trace_name, phone_type = trace
        calc_residuals(trace_name, phone_type)

In [ ]:
main()

Analyzing trace  2020-05-14-US-MTV-1 Pixel4XLModded ...
Following problems detected:
['176 rows with too large BiasUncertaintyNanos', '3 rows with too large ReceivedSvTimeUncertaintyNanos']
Successfully loaded  19211  measurements
Successfully loaded  3571  android fixes
Successfully loaded  23771  ground truth points.
e:  0


/tmp/ipykernel_17720/3377148575.py:26: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  G[ii,:] = [-(x_s - x_0[0])/dist[ii],


Analyzing trace  2020-05-14-US-MTV-1 Pixel4 ...
No problems detected.
Successfully loaded  19444  measurements
Successfully loaded  3584  android fixes
Successfully loaded  23771  ground truth points.
e:  0


/tmp/ipykernel_17720/3377148575.py:26: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  G[ii,:] = [-(x_s - x_0[0])/dist[ii],
